In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import model_from_json
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, UpSampling2D,BatchNormalization
import random
import glob
import subprocess
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, figure

In [ ]:
#we are using colab thats why mounted drive please dont run this cell if you directly take data from system
from google.colab import drive
drive.mount('/content/drive')

we are using colab thats why given this location after mounting drive please provide the folder as per your system.
 we have taken image in batch in this manner like we have provided 45 image from tir by ISRO so I have taken 1 t0 10 image in a batch 1st and 2 to 11 image
 to batch 2nd and so on 

In [ ]:
#initialize wandb and download dataset

num_epochs= 10  
batch_size= 1
height=984           #image height
width=1074           #image width 
n_frames=9           #number of frames 


##########
#we are using colab thats why given this location after mounting drive please provide the folder as per your system
# we have taken image in batch in this manner like we have provided 45 image from tir by ISRO so I have taken 1 t0 10 image ina batch 1 and 2 to 11 image
# to batch 2nd
val_dir='/content/drive/My Drive/catz/ISRO/val_10val'
train_dir='/content/drive/My Drive/catz/ISRO/train_10test'


###########     testing path
#val_dir='/content/drive/My Drive/catz/website_data/23'
############


This function is used for generating 9 images in a sequence and
 1 image for as a output(used for comparing with predicted image)
 and image is already normelize by 255 during data preprocessing and saved in drive link https://drive.google.com/drive/folders/1Y3gkbZvY4PGr1RqqSq8-SUXcLoGViuWx?usp=sharing

In [ ]:
# generator to loop over train and test images
'''
This function is used for generating 9 images in a sequence and
 1 image for as a output(used for comparing with predicted image)
 and image is already normelize by 255 during data preprocessing and saved in drive link
'''
def my_generator3(batch_size, img_dir):
    """A generator that returns 9 images plus a result image"""
    cat_dirs = glob.glob(img_dir + "/*")
    counter = 0
    while True:
        input_images = np.zeros(
            (batch_size,height, width,9*1))
        output_images = np.zeros((batch_size,height, width,1))
        random.shuffle(cat_dirs)
        if (counter+batch_size >= len(cat_dirs)):
            counter = 0
        for i in range(batch_size):
            input_imgs = glob.glob(cat_dirs[counter+i] + "/[0-9]*")
            imgs=[cv2.imread(img,-1).astype(np.float32) for img in sorted(input_imgs)]
            imgs1=[np.reshape(im,(height,width,1)) for im in imgs]
          
            input_images[i]=np.concatenate((imgs1),axis=2)
            output_images[i] =np.reshape((cv2.imread((cat_dirs[counter+i]+"/result.tif"),-1).astype(np.float32)),(height,width,1))            
            #input_images[i] /= 255.
            #output_images[i] /= 255.            
            #input_images[i] /= 4.   #image normalization
            #output_images[i] /= 4.
        yield (input_images, output_images)
        counter =counter+batch_size
        
steps_per_epoch = len(glob.glob(train_dir + "/*")) // batch_size
validation_steps = len(glob.glob(val_dir + "/*")) // batch_size

In [ ]:
#callback to log the images

class ImageCallback(Callback):
    def on_epoch_end(self, epoch, logs):
        validation_X, validation_y = next(
            my_generator3(batch_size, val_dir))
        output = self.model.predict(validation_X)
        """wandb.log({
            "input": [wandb.Image(np.concatenate(np.split(c, 5, axis=2), axis=1)) for c in validation_X],
            "output": [wandb.Image(np.concatenate([validation_y[i], o], axis=1)) for i, o in enumerate(output)]
        }, commit=False)"""

In [ ]:
# Test the generator
test_img='/content/drive/My Drive/catz/ISRO/test_10test'
gen = my_generator3(1, test_img)
videos, next_frame = next(gen)
print(videos[0].shape)
print(next_frame[0].shape)

In [ ]:
#Showing all the images for test
figure()
imshow(videos[0][:,:,0],cmap='gray')
figure()
imshow(videos[0][:,:,1],cmap='gray')
figure()
imshow(videos[0][:,:,2],cmap='gray')
figure()
imshow(videos[0][:,:,3],cmap='gray')
figure()
imshow(videos[0][:,:,4],cmap='gray')
figure()
imshow(videos[0][:,:,5],cmap='gray')
figure()
imshow(videos[0][:,:,6],cmap='gray')
figure()
imshow(videos[0][:,:,7],cmap='gray')
figure()
imshow(videos[0][:,:,8],cmap='gray')
figure()
imshow(next_frame[0][:,:,0],cmap='gray')
# #

#Modal by 50 epoch we have saved model 

In [ ]:
# Conv2DLSTM with Gaussian Noise

from tensorflow.keras.layers import Lambda, Reshape, Permute, Input, add, Conv3D, GaussianNoise, ConvLSTM2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1,l2
def slice(x):
    return x[:,:,:,:, -1]



inp = Input((height, width, n_frames * 1))
reshaped = Reshape((height,width,n_frames,1))(inp)
permuted = Permute((1,2,4,3))(reshaped)
noise = GaussianNoise(0.1)(permuted)
last_layer = Lambda(slice, input_shape=(width,height,next_frame,1), output_shape=(width,height,1))(noise)
permuted_2 = Permute((4,1,2,3))(noise)

conv_lstm_output_1 = ConvLSTM2D(16, (3,3), padding='same',activity_regularizer=l2(0.0001),return_sequences=True)(permuted_2)
conv_lstm_output_1=(BatchNormalization())(conv_lstm_output_1)
conv_lstm_output_1=Dropout(.2)(conv_lstm_output_1)


conv_lstm_output_1 = ConvLSTM2D(32, (3,3), padding='same',activity_regularizer=l2(0.0001),return_sequences=True)(conv_lstm_output_1)
conv_lstm_output_1=(BatchNormalization())(conv_lstm_output_1)
conv_lstm_output_1=Dropout(.2)(conv_lstm_output_1)

conv_lstm_output_1 = ConvLSTM2D(64, (3,3), padding='same',activity_regularizer=l2(0.00001),dilation_rate=2)(conv_lstm_output_1)
conv_lstm_output_1=(BatchNormalization())(conv_lstm_output_1)
conv_lstm_output_1=Dropout(.2)(conv_lstm_output_1)


conv_lstm_output_1 = Conv2D(3, (3,3), padding="same",activity_regularizer=l2(0.00001))(conv_lstm_output_1)
conv_output = Conv2D(1, (3,3), padding="same")(conv_lstm_output_1)
combined = add([last_layer, conv_output])
model=Model(inputs=[inp], outputs=[combined])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit_generator(my_generator3(batch_size*4, train_dir),
                    steps_per_epoch=steps_per_epoch//4,
                    epochs=num_epochs*5, callbacks=[  
    ImageCallback()],
    validation_steps=validation_steps//4,
    validation_data=my_generator3(batch_size*2, val_dir))

In [ ]:
#save model in drive
json_file=model.to_json()
#path=
with open("model_by50epoc","w") as file:
  file.write(json_file)
#weight to HDF5
model.save_weights("model_by50epoc.h5") 


In [ ]:
#predicting next frame from previous 9 frames
result=model.predict(videos)

In [ ]:
# showing predicted image
figure()
imshow(result[0][:,:,0],cmap='gray')
#actual image
figure()
y_true=next_frame[0][:,:,0]
imshow(y_true,cmap='gray')

In [ ]:
y_true=next_frame[0][:,:,0]*255
y_true1=videos[0][:,:,8]*255

y_pred=(result[0][:,:,0])*255
print(y_true.shape,y_pred.shape)


#loss=np.mean(np.square(y_true,y_pred),axis=1)
mse=np.mean(((y_true)-(y_pred))**2)
mse1=np.mean(((y_true1)-(y_pred))**2)
rmse=np.sqrt(np.mean((y_true-y_pred)**2))
rmse1=np.sqrt(np.mean((y_true1-y_pred)**2))
print("rmse,mse with actual frame",rmse,mse)
print("\nrmse1,mse1 with previous frame",rmse1,mse1)

#Modal by 100 epoch

In [ ]:
# Conv2DLSTM with Gaussian Noise

from tensorflow.keras.layers import Lambda, Reshape, Permute, Input, add, Conv3D, GaussianNoise, ConvLSTM2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1,l2
def slice(x):
    return x[:,:,:,:, -1]



inp = Input((height, width, n_frames * 1))
reshaped = Reshape((height,width,n_frames,1))(inp)
permuted = Permute((1,2,4,3))(reshaped)
noise = GaussianNoise(0.1)(permuted)
last_layer = Lambda(slice, input_shape=(width,height,next_frame,1), output_shape=(width,height,1))(noise)
permuted_2 = Permute((4,1,2,3))(noise)

conv_lstm_output_1 = ConvLSTM2D(16, (3,3), padding='same',activity_regularizer=l2(0.0001),return_sequences=True)(permuted_2)
conv_lstm_output_1=(BatchNormalization())(conv_lstm_output_1)
conv_lstm_output_1=Dropout(.2)(conv_lstm_output_1)


conv_lstm_output_1 = ConvLSTM2D(32, (3,3), padding='same',activity_regularizer=l2(0.0001),return_sequences=True)(conv_lstm_output_1)
conv_lstm_output_1=(BatchNormalization())(conv_lstm_output_1)
conv_lstm_output_1=Dropout(.2)(conv_lstm_output_1)

conv_lstm_output_1 = ConvLSTM2D(64, (3,3), padding='same',activity_regularizer=l2(0.00001),dilation_rate=2)(conv_lstm_output_1)
conv_lstm_output_1=(BatchNormalization())(conv_lstm_output_1)
conv_lstm_output_1=Dropout(.2)(conv_lstm_output_1)


conv_lstm_output_1 = Conv2D(3, (3,3), padding="same",activity_regularizer=l2(0.00001))(conv_lstm_output_1)
conv_output = Conv2D(1, (3,3), padding="same")(conv_lstm_output_1)
combined = add([last_layer, conv_output])
model=Model(inputs=[inp], outputs=[combined])


In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit_generator(my_generator3(batch_size*2, train_dir),
                    steps_per_epoch=steps_per_epoch//4,
                    epochs=num_epochs*10, callbacks=[  
    ImageCallback()],
    validation_steps=validation_steps//4,
    validation_data=my_generator3(batch_size*2, val_dir))

In [ ]:
#save model in drive
json_file=model.to_json()
#path=
with open("model_by100epoc","w") as file:
  file.write(json_file)
#weight to HDF5
model.save_weights("model_by100epoc.h5") 


In [ ]:
file=open("model_by50epoc",'r')
model_json=file.read()
file.close()

model=model_from_json(model_json)

# #load weights
model.load_weights("model_by50epoc.h5")

In [ ]:
#predicting next frame from previous 9 frames
result=model.predict(videos)

In [ ]:
# showing predicted image
figure()
imshow(result[0][:,:,0],cmap='gray')
#actual image
figure()
y_true=next_frame[0][:,:,0]
imshow(y_true,cmap='gray')

In [ ]:
y_true=next_frame[0][:,:,0]*255
y_true1=videos[0][:,:,8]*255

y_pred=(result[0][:,:,0])*255
print(y_true.shape,y_pred.shape)


#loss=np.mean(np.square(y_true,y_pred),axis=1)
mse=np.mean(((y_true)-(y_pred))**2)
mse1=np.mean(((y_true1)-(y_pred))**2)
rmse=np.sqrt(np.mean((y_true-y_pred)**2))
rmse1=np.sqrt(np.mean((y_true1-y_pred)**2))
print("rmse,mse with actual frame",rmse,mse)
print("\nrmse1,mse1 with previous frame",rmse1,mse1)

In [ ]:
y_true=next_frame[0][:,:,0]
y_true1=videos[0][:,:,8]

y_pred=(result[0][:,:,0])
print(y_true.shape,y_pred.shape)


mse=np.mean(((y_true)-(y_pred))**2)
mse1=np.mean(((y_true1)-(y_pred))**2)
rmse=np.sqrt(np.mean((y_true-y_pred)**2))
rmse1=np.sqrt(np.mean((y_true1-y_pred)**2))
print("rmse,mse with actual frame",rmse,mse)
print("\nrmse1,mse1 with previous frame",rmse1,mse1)

In [ ]:
save_dir='/content/drive/My Drive/catz/ISRO'
print(save_dir)
cv2.imwrite(os.path.join(save_dir,'pred1.tif'),result[0][:,:,0])

In [ ]:
im=Image.open("/content/drive/My Drive/catz/ISRO/pred1.tif")
im=np.array(im)
imshow(im,cmap='gray')